In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from scipy import stats
from scipy.stats import norm, skew

In [3]:
import seaborn as sb

In [4]:
#EDA

In [5]:
#possible Missing values
miss_val=['NaN']

In [6]:
# Reading Data

In [7]:
aisles=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/aisles.csv', na_values=miss_val)

In [8]:
departments=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/departments.csv', na_values=miss_val)

In [9]:
products=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/products.csv', na_values=miss_val)
#products.isnull().values.any()
#products.head(10)

In [10]:
order_products_prior=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/order_products__prior.csv', na_values=miss_val)

In [11]:
order_products_train=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/order_products__train.csv', na_values=miss_val)

In [12]:
orders=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/orders.csv', na_values=miss_val)
orders.fillna(-1, inplace=True) # Fill missing values with -1

In [13]:
# Combining dataframes orders_prior and orders_train
order_products_total=pd.concat([order_products_prior, order_products_train], axis=0)
order_products_total.shape

(33819106, 4)

In [14]:
order_products_usr=pd.merge(orders, order_products_total, on='order_id')
#order_products_usr.isnull().sum()
order_products_usr.head(10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,-1.0,196,1,0
1,2539329,1,prior,1,2,8,-1.0,14084,2,0
2,2539329,1,prior,1,2,8,-1.0,12427,3,0
3,2539329,1,prior,1,2,8,-1.0,26088,4,0
4,2539329,1,prior,1,2,8,-1.0,26405,5,0
5,2398795,1,prior,2,3,7,15.0,196,1,1
6,2398795,1,prior,2,3,7,15.0,10258,2,0
7,2398795,1,prior,2,3,7,15.0,12427,3,1
8,2398795,1,prior,2,3,7,15.0,13176,4,0
9,2398795,1,prior,2,3,7,15.0,26088,5,1


In [15]:
#Most ordered product by all users
most_freq_prod=order_products_usr.groupby('product_id').size().sort_values(ascending=False)
print(most_freq_prod)

product_id
24852    491291
13176    394930
21137    275577
21903    251705
47209    220877
          ...  
7626          1
14485         1
43098         1
11254         1
36233         1
Length: 49685, dtype: int64


In [16]:
# Total number of times each product was bought by every user
freq_user_product=dict()

for i in range(1,2): #for i in range (1, order_products_usr['user_id'].max()+1):
    freq_user_product=order_products_usr.groupby('user_id')['product_id'].value_counts()
    #freq_user_product[i]=order_products_usr[order_products_usr['user_id']==i]['product_id'].value_counts()
    #most_freq_product_user[i]=freq_user_product[i].index[0]
    #print("Most frequent product for User {} is {}\n\n".format(i,most_freq_product_user[i]))

print(freq_user_product)

user_id  product_id
1        196           11
         10258         10
         12427         10
         25133          9
         13032          4
                       ..
206209   41665          1
         42606          1
         44325          1
         48370          1
         48697          1
Name: product_id, Length: 13863746, dtype: int64


In [17]:
# Total number of unique orders for every user
total_orders_user=order_products_usr.groupby('user_id')['order_id'].nunique()
print(total_orders_user)

user_id
1         11
2         15
3         12
4          5
5          5
          ..
206205     4
206206    67
206207    16
206208    49
206209    14
Name: order_id, Length: 206209, dtype: int64


In [18]:
# This gives the total number of products in EACH order against all users.
# Segregation of the below values.
total_orders_user3=order_products_usr.groupby('user_id')['order_id'].value_counts() 
print(total_orders_user3)

user_id  order_id
1        1187899     11
         2550362      9
         431534       8
         2295261      6
         2398795      6
                     ..
206209   272231       8
         1854736      8
         2307371      3
         2558525      3
         3186442      2
Name: order_id, Length: 3346083, dtype: int64


In [19]:
# This gives the total number of products in all orders for all users. Sum of values above
total_orders_user2=order_products_usr.groupby('user_id')['order_id'].size() 
print(total_orders_user2)

user_id
1          70
2         226
3          88
4          18
5          46
         ... 
206205     51
206206    285
206207    223
206208    677
206209    137
Name: order_id, Length: 206209, dtype: int64


In [20]:
torders=0
for key, value in total_orders_user3.items():
    if (key[0]==1):
        torders+=value

print(torders)

70


In [21]:
#Number of unique products ordered by every user
unique_product_user=dict()

for i in range(1,2): #for i in range (1, order_products_usr['user_id'].max()+1):
    unique_product_user=order_products_usr.groupby('user_id')['product_id'].nunique()

print(unique_product_user)    

user_id
1          19
2         121
3          33
4          17
5          28
         ... 
206205     37
206206    150
206207     92
206208    198
206209     72
Name: product_id, Length: 206209, dtype: int64


In [22]:
#new_product_ratio for every user= unique products/total products
nd=pd.merge(unique_product_user, total_orders_user2, on="user_id")
nd.columns=['unique_product_count', 'total_product_count']
nd['new_product_ratio'] = nd.apply(lambda row: row.unique_product_count/row.total_product_count*100, axis=1)
print(nd)

         unique_product_count  total_product_count  new_product_ratio
user_id                                                              
1                          19                   70          27.142857
2                         121                  226          53.539823
3                          33                   88          37.500000
4                          17                   18          94.444444
5                          28                   46          60.869565
...                       ...                  ...                ...
206205                     37                   51          72.549020
206206                    150                  285          52.631579
206207                     92                  223          41.255605
206208                    198                  677          29.246677
206209                     72                  137          52.554745

[206209 rows x 3 columns]


In [23]:
#Product ordering ratio for every user = total number of orders containing product/ total number of orders
#order_contains_product=order_products_usr.groupby(['user_id', 'product_id'])['order_id'].count()
order_contains_product=order_products_usr.groupby(['user_id', 'product_id'])['order_id'].agg(['count']).reset_index().rename(columns={'count': "orders_with_product"})
total_user_order=order_products_usr.groupby('user_id')['order_id'].agg(['count']).reset_index().rename(columns={'count': "total_orders"})
nro=pd.merge(order_contains_product, total_user_order, on="user_id")
nro['product_order_ratio']=nro.apply(lambda row: row.orders_with_product/row.total_orders*100, axis=1)
print(nro)

KeyboardInterrupt: 

In [24]:
print(total_user_order.iloc[0]) #iloc takes specific row number as paramter.

user_id          1
total_orders    70
Name: 0, dtype: int64


In [25]:
# Frequency of products ordered by every user in each order
#user_order_products22=order_products_usr.groupby(['user_id', 'order_id'])['product_id'].count() # returns a Series
user_order_products=order_products_usr.groupby(['user_id', 'order_id'])['product_id'].agg(['count',  'mean']) # returns a Dataframe
print(user_order_products)

                  count          mean
user_id order_id                     
1       431534        8  16303.125000
        473747        5  15692.800000
        550135        5  12209.200000
        1187899      11  27538.727273
        2254736       5  14883.800000
...                 ...           ...
206209  2307371       3   9809.000000
        2558525       3  22968.333333
        2977660       9  22614.666667
        3154581      13  21595.846154
        3186442       2  10192.000000

[3346083 rows x 2 columns]


In [26]:
for a in user_order_products.itertuples():
    print(a[0], a[1], a[2])

(1, 431534) 8 16303.125
(1, 473747) 5 15692.8
(1, 550135) 5 12209.2
(1, 1187899) 11 27538.727272727272
(1, 2254736) 5 14883.8
(1, 2295261) 6 23899.666666666668
(1, 2398795) 6 12529.5
(1, 2539329) 5 15840.0
(1, 2550362) 9 24636.777777777777
(1, 3108588) 6 23899.666666666668
(1, 3367565) 4 12003.5
(2, 738281) 13 32927.769230769234
(2, 788338) 16 26128.3125
(2, 839880) 16 21086.6875
(2, 1199898) 21 29012.238095238095
(2, 1402090) 15 20064.4
(2, 1447487) 9 16194.0
(2, 1492625) 31 24592.16129032258
(2, 1501582) 6 24626.166666666668
(2, 1673511) 13 25888.46153846154
(2, 1718559) 26 22606.30769230769
(2, 1901567) 5 27557.4
(2, 2168274) 13 29251.0
(2, 3186735) 19 22897.42105263158
(2, 3194192) 14 31647.64285714286
(2, 3268552) 9 27211.555555555555
(3, 444309) 9 22692.11111111111
(3, 521107) 5 34097.6
(3, 676467) 6 26105.0
(3, 1374495) 10 26498.8
(3, 1402502) 6 29319.666666666668
(3, 1839752) 9 21184.222222222223
(3, 1972919) 8 25585.25
(3, 2037211) 5 13005.2
(3, 2710558) 11 28055.0
(3, 3002854

(64, 2754807) 11 33283.09090909091
(64, 3129760) 20 28435.5
(64, 3215917) 17 29283.0
(65, 225812) 8 18064.125
(65, 692225) 7 23272.285714285714
(65, 864720) 17 23479.470588235294
(65, 1072954) 8 22796.0
(65, 1422896) 11 22447.727272727272
(65, 1583644) 16 27871.625
(65, 1625976) 13 18667.53846153846
(65, 1999119) 6 21649.0
(65, 2028484) 3 25697.666666666668
(65, 2133732) 8 19269.375
(65, 2630919) 10 23363.3
(65, 2843095) 5 28953.8
(65, 3003586) 8 24099.75
(65, 3029625) 7 26518.0
(65, 3253911) 13 25714.923076923078
(66, 4595) 5 20353.6
(66, 34647) 7 28676.14285714286
(66, 932035) 5 24728.0
(66, 1089019) 7 31692.428571428572
(66, 2143319) 3 16655.666666666668
(66, 2360283) 1 28842.0
(66, 2401332) 2 30335.5
(67, 29758) 3 18842.0
(67, 46584) 1 35561.0
(67, 150528) 5 21486.8
(67, 551869) 2 3190.0
(67, 596585) 3 6518.666666666667
(67, 768572) 3 18099.0
(67, 863974) 3 19049.0
(67, 970922) 6 20146.833333333332
(67, 1515698) 4 4921.5
(67, 1558976) 6 26665.0
(67, 1568481) 3 8982.333333333334
(67

(138, 1069065) 4 44734.75
(138, 1154289) 2 26851.5
(138, 1421595) 6 25121.833333333332
(138, 1433006) 3 30439.0
(138, 1512043) 5 26659.0
(138, 1592314) 4 16727.0
(138, 1593782) 4 26527.25
(138, 1854193) 3 32658.333333333332
(138, 1898191) 3 28129.666666666668
(138, 1972635) 4 33224.25
(138, 1977647) 4 15344.5
(138, 1986630) 6 25361.166666666668
(138, 2027699) 2 31296.0
(138, 2138731) 4 25333.75
(138, 2206168) 2 21475.5
(138, 2226790) 6 23544.5
(138, 2254091) 5 27851.0
(138, 2329217) 6 28199.166666666668
(138, 2526903) 9 27039.666666666668
(138, 2823056) 7 23701.428571428572
(138, 3052988) 6 17545.5
(138, 3139998) 5 18922.8
(138, 3160996) 8 19151.875
(139, 1296919) 2 20918.0
(139, 1515111) 6 16754.833333333332
(139, 2233425) 3 32115.0
(139, 2610428) 3 28396.0
(139, 3220002) 3 26373.333333333332
(140, 22418) 23 18167.304347826088
(140, 385418) 12 26180.666666666668
(140, 405367) 17 22633.41176470588
(140, 412724) 6 27325.833333333332
(140, 414277) 19 24419.315789473683
(140, 458750) 22 2

(195, 1113934) 3 18754.0
(195, 1190066) 3 18754.0
(195, 1199792) 4 26945.0
(195, 1205026) 3 30250.0
(195, 1302290) 2 38900.0
(195, 1357499) 1 31651.0
(195, 1442427) 1 31651.0
(195, 1458129) 6 19493.0
(195, 1485596) 4 17374.0
(195, 1615297) 2 23172.5
(195, 1680569) 3 26034.666666666668
(195, 1684209) 5 25846.0
(195, 1738718) 3 20503.0
(195, 1765587) 3 20503.0
(195, 1896889) 4 24684.25
(195, 1946131) 3 28154.666666666668
(195, 1965520) 6 32166.666666666668
(195, 1991051) 1 31651.0
(195, 2040988) 2 23172.5
(195, 2173254) 6 28394.5
(195, 2186781) 3 28595.666666666668
(195, 2213852) 4 30329.5
(195, 2247949) 3 22402.666666666668
(195, 2254988) 2 27219.5
(195, 2257634) 5 18024.4
(195, 2443791) 2 6129.5
(195, 2500838) 3 27881.666666666668
(195, 2661660) 2 22413.5
(195, 2699449) 2 38118.0
(195, 2744976) 2 23172.5
(195, 2779493) 4 21860.75
(195, 2781919) 2 23172.5
(195, 2857448) 4 21860.75
(195, 2925383) 2 28619.5
(195, 2969331) 4 35858.75
(195, 3058369) 3 20503.0
(195, 3094213) 2 39141.5
(195, 

(248, 175329) 8 15575.5
(248, 190149) 3 34605.333333333336
(248, 222790) 5 31482.6
(248, 398960) 8 23550.75
(248, 445804) 5 20604.0
(248, 495322) 15 26423.866666666665
(248, 650872) 5 31507.2
(248, 805722) 7 30181.285714285714
(248, 838358) 9 26147.777777777777
(248, 920620) 6 22545.5
(248, 1219005) 6 31311.833333333332
(248, 1230903) 6 34102.5
(248, 1320795) 5 29981.2
(248, 1343011) 13 25166.76923076923
(248, 1401922) 6 34041.333333333336
(248, 1475084) 11 27428.454545454544
(248, 1625970) 4 28966.25
(248, 1657793) 9 28213.222222222223
(248, 1766555) 13 24717.53846153846
(248, 1789225) 13 23589.384615384617
(248, 1841261) 5 33268.2
(248, 1951775) 5 27733.8
(248, 1975496) 9 26636.444444444445
(248, 2002675) 6 30497.166666666668
(248, 2018221) 8 24224.375
(248, 2052479) 5 28949.2
(248, 2055081) 8 27604.375
(248, 2065169) 3 33988.666666666664
(248, 2081151) 5 20964.2
(248, 2144600) 13 25006.153846153848
(248, 2268775) 6 20527.166666666668
(248, 2269056) 16 24310.25
(248, 2285295) 2 22379

...

(5715, 3294320) 9 22515.88888888889
(5715, 3395231) 11 22038.81818181818
(5716, 929433) 15 26464.533333333333
(5716, 1194620) 11 30875.636363636364
(5716, 2085060) 11 22551.363636363636
(5716, 2262273) 15 34083.13333333333
(5716, 2805257) 7 29054.0
(5716, 3006349) 13 32440.23076923077
(5716, 3104820) 12 26498.333333333332
(5716, 3159715) 7 22793.85714285714
(5716, 3222152) 15 26464.533333333333
(5716, 3378566) 15 25052.533333333333
(5717, 322953) 16 24703.0
(5717, 824386) 8 16200.75
(5717, 1026541) 12 20966.083333333332
(5717, 1061414) 12 20695.25
(5717, 1345810) 19 26999.736842105263
(5717, 1383625) 11 21809.727272727272
(5717, 1389471) 14 27716.571428571428
(5717, 2275111) 9 16211.444444444445
(5717, 2329587) 22 23674.772727272728
(5717, 3265622) 13 16526.76923076923
(5717, 3385211) 26 23372.80769230769
(5718, 4853) 2 21736.5
(5718, 115020) 3 17434.0
(5718, 117348) 6 21249.333333333332
(5718, 244310) 1 10473.0
(5718, 347549) 2 38346.0
(5718, 428951) 7 28331.85714285714
(5718, 1091615

(5760, 3062001) 3 24140.666666666668
(5760, 3078953) 10 26507.4
(5760, 3115763) 1 14233.0
(5760, 3167897) 1 14233.0
(5760, 3197582) 7 21998.14285714286
(5760, 3205034) 2 10654.5
(5761, 317201) 2 19330.5
(5761, 709709) 1 9510.0
(5761, 728902) 3 18764.333333333332
(5761, 785046) 1 19348.0
(5761, 858555) 5 22882.2
(5761, 1014070) 5 21664.6
(5761, 1060121) 3 16853.333333333332
(5761, 1190979) 2 13650.0
(5761, 1655499) 4 21011.75
(5761, 1810367) 1 8536.0
(5761, 1935256) 1 36011.0
(5761, 2151448) 3 22230.0
(5761, 2342424) 5 24753.4
(5761, 2423287) 4 17577.0
(5761, 2498776) 5 20615.0
(5761, 2811344) 5 15042.4
(5761, 2941229) 1 20076.0
(5761, 2961090) 2 10142.0
(5761, 3352957) 8 29541.75
(5762, 25032) 36 22571.472222222223
(5762, 187964) 17 21849.529411764706
(5762, 217234) 33 20218.303030303032
(5762, 259423) 20 23984.1
(5762, 311986) 35 24130.6
(5762, 375930) 19 24111.63157894737
(5762, 435798) 30 20866.433333333334
(5762, 541218) 2 42333.5
(5762, 667271) 9 19553.777777777777
(5762, 910454) 

(5820, 520142) 19 26126.526315789473
(5820, 1850572) 16 20932.125
(5820, 2757726) 10 22964.9
(5820, 3176344) 22 21278.636363636364
(5821, 1410266) 1 17924.0
(5821, 3037302) 3 25841.666666666668
(5821, 3226267) 2 20221.0
(5822, 524338) 9 23651.333333333332
(5822, 536151) 4 24438.25
(5822, 1626086) 15 27093.866666666665
(5822, 2093342) 8 25283.125
(5822, 2254331) 17 32229.647058823528
(5822, 3332805) 10 30276.1
(5823, 425978) 4 43661.5
(5823, 641652) 3 26814.0
(5823, 1027420) 5 13383.4
(5823, 1091846) 9 27936.11111111111
(5823, 1175995) 12 36889.5
(5823, 2282068) 5 18733.4
(5823, 2301418) 10 23865.4
(5823, 2388301) 4 28926.0
(5823, 2479237) 9 27936.11111111111
(5823, 2677990) 6 16641.833333333332
(5823, 3410516) 4 24478.0
(5824, 8242) 10 31328.7
(5824, 120044) 14 23496.214285714286
(5824, 129585) 4 29442.75
(5824, 167026) 18 29054.333333333332
(5824, 196516) 7 20272.428571428572
(5824, 472064) 14 21169.0
(5824, 493326) 12 22985.75
(5824, 847242) 12 25151.75
(5824, 851905) 6 21065.8333333

(5877, 68713) 18 24969.722222222223
(5877, 155849) 38 26447.026315789473
(5877, 199953) 26 23655.923076923078
(5877, 255029) 16 22565.9375
(5877, 420736) 24 24787.416666666668
(5877, 565674) 22 26648.727272727272
(5877, 609246) 43 27519.13953488372
(5877, 728694) 25 28574.44
(5877, 737511) 20 27163.75
(5877, 838133) 28 26883.821428571428
(5877, 869136) 15 20623.266666666666
(5877, 898177) 24 23206.666666666668
(5877, 1020820) 16 23536.4375
(5877, 1485118) 31 22485.354838709678
(5877, 1668130) 6 32254.833333333332
(5877, 2220145) 21 25403.0
(5877, 2643298) 9 26405.222222222223
(5877, 2996843) 23 25938.652173913044
(5877, 3013517) 14 27006.714285714286
(5878, 388051) 4 17356.25
(5878, 1218374) 7 30853.571428571428
(5878, 1226567) 4 35733.5
(5878, 1549589) 9 17813.777777777777
(5878, 1800454) 10 21935.4
(5878, 1809754) 9 22390.11111111111
(5878, 2555127) 4 25446.5
(5879, 312286) 9 22661.222222222223
(5879, 716707) 10 33175.4
(5879, 778742) 10 20141.7
(5879, 1305839) 10 22418.7
(5879, 1961

KeyboardInterrupt: 

In [44]:
# Most Frequent time for every user
user_time=order_products_usr.groupby('user_id')['order_hour_of_day'].value_counts()

In [43]:
#Most frequent Time of purchase for the shop
most_popular_time=order_products_usr.groupby('order_hour_of_day')['user_id'].count()

In [45]:
# Most frequent day of purchase for every user
user_day=order_products_usr.groupby('user_id')['order_dow'].value_counts()

In [42]:
# Most frequent day of purchase for shop
most_popular_day=order_products_usr.groupby('order_dow')['user_id'].count()

In [31]:
#How many times a user has reordered a product
user_products_NotReordered=order_products_usr.groupby(['user_id', 'product_id'])['reordered'].aggregate("count").reset_index().rename(columns={'reordered':'reordered_times'})
print (user_products_NotReordered)

          user_id  product_id  reordered_times
0               1         196               11
1               1       10258               10
2               1       10326                1
3               1       12427               10
4               1       13032                4
...           ...         ...              ...
13863741   206209       43961                3
13863742   206209       44325                1
13863743   206209       48370                1
13863744   206209       48697                1
13863745   206209       48742                2

[13863746 rows x 3 columns]


In [ ]:
#Total number of times a product has been reordered.
#percentage of user wrt product

In [ ]:
#Overall percentage of reorders for each product

In [ ]:
#Most ordered product in every department

In [ ]:
#Did any user reorder their previous order

In [ ]:
#Most frequent time when users purchase

In [ ]:
#Times a customer orders from the same department. Wrt to customer

In [ ]:
#Days a customer orders the same products

In [ ]:
#Times a product is ordered from the same department. Wrt product